<a href="https://colab.research.google.com/github/aminehd/AminehTechnicalWritings/blob/main/SamplingStatsAndDistros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Outline
- Introduction (Purpose + scope)
- Body
  - sample summary statistics and examples (coefficients of fitted model)
  - load data and showcase
  - Sampling distribution of summary statistics. (use sample mean as example)
  - Expected value of summary statistics
  - Variance of summary statistic
  - Variance within a sample vs variance of a given stat over samplings
  - Standrad Error of stats and confidence interval. (SE(beta0))
  - F-test (within groups variance vs between group variance)
- Conclusion:

## Git setup and data load

In [1]:
import google.colab
repo_path = 'AminehTechnicalWritings'
!git -C $repo_path pull origin || git clone https://github.com/aminehd/AminehTechnicalWritings/ $repo_path
local_clone = '/content/' + repo_path
!git config --global user.email "amineh.dadsetan@gmail.com"
!git config --global user.name "Amineh Dadsetan"

fatal: cannot change to 'AminehTechnicalWritings': No such file or directory
Cloning into 'AminehTechnicalWritings'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 168 (delta 94), reused 139 (delta 79), pack-reused 3
Receiving objects: 100% (168/168), 3.12 MiB | 18.25 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
import numpy as np
files = np.load(local_clone + '/population.npz')
# sample = np.load(local_clone + '/sample.npy'
population = files['arr_0']
population.shape

(300000, 2)

### Summary Statistic

Previously we explained the concept of population and the concept of sample. Basically we have one population (e.g., population of American people) but we can sample a subset in many ways (e.g., sample of America people under age 18, sample of American people living in new your, etc).

In fact if N is population size, there is $2^N$ samples.

Now a statistic or measure can be defined on the whole population as well as any given sample. A statistic or a measure in mathematic is a generalization of concepts such as lenght or heigh for a given set. Since samples and the population are sets, we can define such things on them. A well known one is mean. Thus you can think of population mean or any given sample mean.

So far it was fairly easy to explain things. But it gets a bit trickier when we wanna talk the distribution of such measures over allll samples and compare them with the value of measure on the poulation.

The motivation behind sampling is that population is too big and it is virtually impossible to calculate the mean for the whole population. Let's say as a minister of something you need the average age of your population.  

Let's look at the population mean and sample mean for the data set we have created in previous notebooks.There are only 30 distinct x values in the pupolation data. Let pick a subset of such distinct x.




In [10]:
print(population[:, 0])
len(population[:,0])
distinct_xs = np.unique(population[:, 0])
len(distinct_xs)
x_samp_index = np.linspace(0, len(distinct_xs)-1, 10).astype(int)
x_samples = distinct_xs[x_samp_index]
x_samples
n_repeat = len(population[:,0]) // len(distinct_xs)
n_repeat

[ 1.  1.  1. ... 10. 10. 10.]


10000

We sampled 10 distinct x and stored them in `x_samples`. There are several observations for each x in sample and thus several responses in the population. Thus we can have several sampling with given `x_samples`. Let's pick one such and plot it:
Next thing is you wanna randomly pick y's that are response to these x's. Not easiest to implement now. think of abstraction HERE!!!

In [13]:
rand_gen = np.random.default_rng(2)
def sample_response(x_sams, population, rand_gen):
  for x in x_sams:
    # generate integer between 0 to n_repeat
    index = rand_gen.integers(0, n_repeat)
    yield (population[x * n_repeat + index, 0], population[x * n_repeat + index, 1],)


print(list(sample_response(x_samp_index, population, rand_gen)))
print(x_samples)
# YAY HERE!! plot the sample distro and mark it's mean with red

[(1.0, 4.932189294678029), (1.9310344827586208, 7.716893210378157), (2.8620689655172415, 10.628354600610912), (3.793103448275862, 12.406025998951204), (4.724137931034483, 12.387762801704252), (5.9655172413793105, 14.347471279766285), (6.8965517241379315, 18.976850098018463), (7.827586206896552, 18.31943450530723), (8.758620689655173, 20.14873201884266), (10.0, 24.278163704720317)]
[ 1.          1.93103448  2.86206897  3.79310345  4.72413793  5.96551724
  6.89655172  7.82758621  8.75862069 10.        ]




https://en.wikipedia.org/wiki/Sampling_distribution
start with mu but get to beta_i HERE!!!

The concept of unbiased estimator pops up one more time. HERE!!!


Then we look at expected value of sample statistices over multiple sampling. Also we look at variance of stuff within sample, and variance of sample stat over several samplings. This indeed is an intersting case when sample stat is defined to be the mean of sample. HERE!!!




notebook we try to look at the model coefficients ( 𝛽𝑖 's ) as summary statistics defined on samples. You can also think of them as measures since samples are subsets.



These can help understand concept of Standard Error and F-test better.